<a href="https://colab.research.google.com/github/Benjamin-morel/TensorFlow/blob/main/03_%5BRP%5D_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


# **Machine Learning Model: regression and predictions**
| | |
|------|------|
| Filename | 03_[RP]_regression.ipynb |
| Author(s) | Benjamin Morel (benjaminmorel27@gmail.com) |
| Date | September 8, 2024 |
| Aim(s) | Do regression tasks with a neural network machine learning model|
| Dataset(s) | Auto MPG dataset from UC Irvine [[1]](https://archive.ics.uci.edu/dataset/9/auto+mpg) |
| Version | Python 3.10.12 - TensorFlow 2.17.1 - final notebook version |


<br> **!!Read before running!!** <br>
1. Fill in the inputs
2. CPU execution is enough
3. Run all and read comments.


---

#### **Motivation**

With the Auto MPG datase from the UC Irvine university [[1]](https://archive.ics.uci.edu/dataset/9/auto+mpg), a neural network model is created to predict fuel efficiency of any 1970s / 1980s automobiles. Four regression models are created and then compared.

#### **Outline**
* retrieve data from a .csv
* cleaning data and pre-processing
* model definition and training
* comparison & predictions
* references



---


## **0. Input section**

The model has already been trained and the user can choose to used a pre-trained model (No) or to repeat the training phase (Yes). Using a pre-trained model saves time, computer resources and CO2 emissions.

In [1]:
training_phase = 'No'

---


## **1.Python librairies & display utilities**

In [2]:
# @title 1.1. Python librairies [RUN ME]

"""math"""
import numpy as np # linear algebra

"""data"""
from pandas import read_csv, DataFrame # data manipulation tool

"""ML models"""
import tensorflow as tf # framework for ML/DL
from tensorflow import keras, optimizers, linspace # API used to build model in TensorFlow

"""display"""
from plotly.graph_objects import Figure, Splom, Scatter, Histogram  # graphing packages
from plotly.subplots import make_subplots # graphing packages

"""performances"""
from time import thread_time, time # timer
start = time()

In [3]:
# @title 1.2. Import Github files [RUN ME]

"""clone the Github repertory TensorFlow and imports the files required (see sections 4 to 8)"""
def get_github_files():
  !git clone https://github.com/Benjamin-morel/TensorFlow.git TensorFlow_duplicata
  path_model_L = 'TensorFlow_duplicata/99_pre_trained_models/03_regression/Lmodel.keras'
  path_model_nL = 'TensorFlow_duplicata/99_pre_trained_models/03_regression/nLmodel.keras'
  path_model_DNN = 'TensorFlow_duplicata/99_pre_trained_models/03_regression/DNNmodel.keras'
  path_model_nDNN = 'TensorFlow_duplicata/99_pre_trained_models/03_regression/nDNNmodel.keras'
  path_csv = 'TensorFlow_duplicata/99_pre_trained_models/03_regression/performances_regression.csv'
  model_L = keras.models.load_model(path_model_L)
  model_nL = keras.models.load_model(path_model_nL)
  model_DNN = keras.models.load_model(path_model_DNN)
  model_nDNN = keras.models.load_model(path_model_nDNN)
  performances_CSV = read_csv(path_csv)
  !rm -rf TensorFlow_duplicata/
  return model_L, model_nL, model_DNN, model_nDNN, performances_CSV

In [4]:
# @title 1.3. Figure plots [RUN ME]

"""plot multi-graph(see section 3.1)"""
def plot_data(dataset):
  df = dataset
  fig = Figure(data=Splom(dimensions=[dict(label='MPG', values=df['MPG']),
                                      dict(label='Displacement', values=df['Displacement']),
                                      dict(label='Horsepower', values=df['Horsepower']),
                                      dict(label='Weight', values=df['Weight'])],
                          diagonal_visible=False)) # remove plots on diagonal

  fig.update_layout(title="Some representations of the dataset", width=1000, height=600)
  fig.show()

"""plot training curves (see sections 4 to 7)"""
def show_evolution(history):
  history_dict = history.history
  history_dict.keys()

  fig = Figure()

  acc_train, acc_val = history_dict['loss'], history_dict['val_loss']
  epochs = np.array(range(1, len(acc_train) + 1))

  fig.add_trace(Scatter(x=epochs, y=acc_train, mode='lines', name='loss'))
  fig.add_trace(Scatter(x=epochs, y=acc_val, mode='lines', name='val_loss'))
  fig.update_layout(legend=dict(x=0.02, y=0.98, xanchor='left', yanchor='top', bgcolor='rgba(255, 255, 255, 0.8)', bordercolor='black', borderwidth=1), width=500, height=500)
  fig.update_xaxes(title = "epochs"), fig.update_yaxes(title = "loss")

  fig.show()

"""plot histogram and performance graph (see section 8)"""
def plot_prediction(name):
  model = all_model[name]
  if name == 'L-model' or name == 'DNN-model':
    predictions = model.predict(test_features["Horsepower"], verbose=0).flatten()
  else:
    predictions = model.predict(test_features, verbose=0).flatten()
  fig = make_subplots(rows=1, cols=2, subplot_titles=["Predicted values Vs. True values", "Error distribution"])
  # Subplot 1
  fig.add_trace(Scatter(x=test_labels, y=predictions, mode='markers', marker_color='blue', showlegend=False), row=1, col=1)
  xaxis = linspace(min(test_labels)*0.9,max(test_labels)*1.1,50)
  fig.add_trace(Scatter(x=xaxis, y=xaxis, mode='lines', marker_color='red', showlegend=False), row=1, col=1) # line y=x
  # Subplot 2
  fig.add_trace(Histogram(x=predictions - test_labels, nbinsx=25, marker_color='blue', showlegend=False), row=1, col=2)
  fig.update_yaxes(title_text="Predictions [MPG]", row=1, col=1), fig.update_yaxes(title_text="Count", range=[0, 25], row=1, col=2)
  # Layout and axis
  fig.update_layout(title=dict(text=name), width=900, height=500)
  fig.update_xaxes(title_text="True values [MPG]", row=1, col=1), fig.update_xaxes(title_text="Prediction error [MPG]", row=1, col=2)
  fig.show()

---


## **2. Data retrieval and set generation**

### 2.1. Data acquisition

The dataset is composed of 9 characteristics of 398 car models, with `mpg` as the target variable.

| | | |
|------|------|------|
| Quantity | Description | Unit |
| displacement | engine displacement | [in^3] |
| cylinders | number of cylinders | - |
| weight | vehicle weight | [lb] |
| model year | car's model year | - |
| origin | 1 = USA, 2 = Europe, 3 = Japan | - |
| car name | descriptive string | - |
| acceleration | time to accelerate from 0 to 60 mph | [s] |
| horsepower | engine power | - |
| mpg | car's fuel efficiency | [mi/gal] |


In [5]:
"""retrieve data"""
def get_data(url):
  raw_dataset = read_csv(url, names=column_names, na_values='?',  comment='\t',  sep=' ', skipinitialspace=True)
  print("Summary of the dataset: ", "\n", "\nNumber of rows in raw_dataset: ", len(raw_dataset), "/ 398", "\nNumber of missing values: ", raw_dataset["Horsepower"].isna().sum(), "/", len(raw_dataset)*9)
  return raw_dataset

In [6]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin']

raw_dataset = get_data(url)

Summary of the dataset:  
 
Number of rows in raw_dataset:  398 / 398 
Number of missing values:  6 / 3582


In [7]:
raw_dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


### 2.2. Dataset cleaning

Vehicle models with missing data - horsepower - are removed.

In [8]:
dataset = raw_dataset.copy() # copy of the original data before modification
dataset = dataset.dropna() # remove rows containing NaN values

print("Summary of the dataset after data cleaning: ", "\n", "\nNumber of rows in the dataset cleaned: ", len(dataset), "/", len(raw_dataset), "\nNumber of missing values: ", dataset["Horsepower"].isna().sum(), "/", len(dataset)*9) # horsepower NaN values

Summary of the dataset after data cleaning:  
 
Number of rows in the dataset cleaned:  392 / 398 
Number of missing values:  0 / 3528


---


## **3. Sample preparation & pre-processing**

### 3.1. Set generation

The dataset is randomly split into 2 subsets: training (80%) and test (20%) [[2]](https://arxiv.org/pdf/2202.03326). Test set is used for a finale evaluation of the model.

In [9]:
train_dataset = dataset.sample(frac=0.8) # randomly draws 80% of dataset
test_dataset = dataset.drop(train_dataset.index) # removes the rows selected for the training set from dataset

print("# of the training set:", len(train_dataset), "\n# of the test set:", len(test_dataset), "\n# of the training and test sets:", len(train_dataset)+len(test_dataset))

# of the training set: 314 
# of the test set: 78 
# of the training and test sets: 392


Fuel efficiency `MPG` is the only variable that is not linearly related to any other variable.

In [10]:
plot_data(train_dataset)

### 3.2. Training set handling

Training set is organized by arranging label variable and feature variables.

In [11]:
train_features, test_features = train_dataset.copy(), test_dataset.copy()
train_labels, test_labels = train_features.pop('MPG'), test_features.pop('MPG')

### 3.3. Normalization step

For the model, `Horsepower` is selected as the variable used to predict `MPG` ( possible to use another variable). Therefore, `Horsepower` is kept in the dataset `train_features` and used to adapt the normalisation layer.


In [12]:
horsepower = np.array(train_features['Horsepower'])
horsepower_normalizer = keras.layers.Normalization(input_shape=(1,), axis=None)
horsepower_normalizer.adapt(horsepower) # fit to the data to compute the mean and standard deviation

/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



---


## **4. Linear regression model (L-model)**

First model: a single-variable linear regression model to predict `MPG` from `Horsepower`. For this, two steps are done in this model:
*   The feature `Horsepower` is normalized by using a specific preprocessing layer `norm` (then replaced by the one defined above).
*   A linear transformation `y = ax + b` is applied with the layer `Dense` to produce 1 output.



In [13]:
"""setup model"""
def create_linear_model(norm):
  model = keras.Sequential([norm, keras.layers.Dense(units=1)])
  model.compile(optimizer=optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error')
  return model

In [14]:
linear_model = create_linear_model(horsepower_normalizer)

if training_phase == 'Yes':
  computation_time = {}
  t0 = thread_time()
  history = linear_model.fit(train_features['Horsepower'], train_labels, epochs=50, verbose=0, validation_split = 0.2)
  t1 = thread_time()
  computation_time['L-model'] = t1-t0 # save the computation time
  linear_model.save('Lmodel.keras')
  show_evolution(history)

In [15]:
if training_phase == "Yes":
  test_results = {}
  test_results['L-model'] = linear_model.evaluate(test_features['Horsepower'], test_labels, verbose=0) # save the loss value for the test set

---


## **5. Multiple inputs linear regression model (nL-model)**

Second model: a 8-variables linear regression model to predict `MPG` from `train_features`. For this, two steps are done in this model:
*   The features `train_features` are normalized by using a specific preprocessing layer `norm`
*   A linear transformation `y = Mx + v` is applied with the layer `Dense` to produce 1 output

In [16]:
normalizer_multiple = keras.layers.Normalization(axis=-1)
normalizer_multiple.adapt(np.array(train_features))

In [17]:
multiple_linear_model = create_linear_model(normalizer_multiple)
if training_phase == "Yes":
  t0 = thread_time()
  history = multiple_linear_model.fit(train_features, train_labels, epochs=50, verbose=0, validation_split = 0.2)
  t1 = thread_time()
  multiple_linear_model.save('nLmodel.keras')
  computation_time['nL-model'] = t1-t0
  test_results['nL-model'] = multiple_linear_model.evaluate(test_features, test_labels, verbose=0) # save the loss value for the test set
  show_evolution(history)

---


## **6. Deep Neural Network model (DNN-model)**

Third model: a deep network regression model to predict `MPG` from `Horsepower`. The model is composed of one pre-processing layer, 2 deep layers and one output layer:
* The pre-processing layer `norm`
* Two deep layers `Dense` with the non-linear function `ReLu` as activation function
* One output layer `Dense` to compute an unique output


In [18]:
"""setup model"""
def DNN_model(norm):
  model = keras.Sequential([
      norm,
      keras.layers.Dense(64, activation='relu'),
      keras.layers.Dense(64, activation='relu'),
      keras.layers.Dense(1)])

  model.compile(loss='mean_absolute_error', optimizer=keras.optimizers.Adam(0.001))
  return model

In [19]:
dnn_model = DNN_model(horsepower_normalizer)
if training_phase == "Yes":
  t0 = thread_time()
  history = dnn_model.fit(train_features['Horsepower'], train_labels, epochs=50, validation_split=0.2, verbose=0)
  t1 = thread_time()
  computation_time['DNN-model'] = t1-t0
  test_results['DNN-model'] = dnn_model.evaluate(test_features['Horsepower'], test_labels, verbose=0)
  dnn_model.save('DNNmodel.keras')
  show_evolution(history)

---


## **7. Multiple inputs deep Neural Network model (nDNN-model)**

Last model: a deep network regression model with 8 inputs to predict `MPG` from `train_features`.

In [20]:
multiple_dnn_model = DNN_model(normalizer_multiple)
if training_phase == "Yes":
  t0 = thread_time()
  history = multiple_dnn_model.fit(train_features, train_labels, epochs=50, validation_split=0.2, verbose=0)
  t1 = thread_time()
  computation_time['nDNN-model'] = t1-t0
  test_results['nDNN-model'] = multiple_dnn_model.evaluate(test_features, test_labels, verbose=0)
  multiple_dnn_model.save('nDNNmodel.keras')
  show_evolution(history)

---


## **8. Confrontation & comments**

If the models have been trained, it is possible to compare the performance of each model. `nDNN_model` is the best in terms of minimizing the loss function, but also one of the longest in terms of computing time. Depending on performance, the function `plot_prediction` can be used to visualize predicted values as a function of true values (left graph) and error distribution (right graph).

In [21]:
if training_phase == "Yes":
  performances = DataFrame([test_results, computation_time], index=['Mean absolute error [MPG]', "Computation time [s]"]).T
  performances.to_csv('performances_regression.csv', index=False)
else:
  linear_model, multiple_linear_model, dnn_model, multiple_dnn_model, performances = get_github_files()

Cloning into 'TensorFlow_duplicata'...
remote: Enumerating objects: 767, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 767 (delta 154), reused 96 (delta 96), pack-reused 583 (from 2)
Receiving objects: 100% (767/767), 186.15 MiB | 26.77 MiB/s, done.
Resolving deltas: 100% (385/385), done.


In [22]:
print(performances)

   Mean absolute error [MPG]  Computation time [s]
0                   3.827374              4.595454
1                   2.092484              3.868038
2                   3.352383              5.078055
3                   1.965180              4.296537


In [23]:
all_model = {'L-model': linear_model, 'nL-model': multiple_linear_model, 'DNN-model': dnn_model, 'nDNN-model':	multiple_dnn_model}

In [24]:
plot_prediction('nDNN-model')

The 4 predictive models produce different results and expectations. The multiple-input neural network `nDNN` model produces the closest predictions to reality with a reasonable computation time. However, with much less complexity, the multi-input linear `nL` model seems more than satisfactory. The prediction error is close to that of the complex `nDNN` model, but with a lower computation time.

**Depending on resources and requirements, these two models are equivalent.**

---


## **9. References**

| | | | | |
|------|------|------|------|------|
| Index | Title | Author(s) | Type | Comments |
|[[1]](https://archive.ics.uci.edu/dataset/9/auto+mpg) | Auto MPG | UC Irvine | dataset | - |
|[[2]](https://arxiv.org/pdf/2202.03326) | Optimal Ratio for Data Splitting | V. Roshan Joseph | paper | - |

In [25]:
print("Notebook run in %.1f seconds on %s" % ((time() - start), tf.config.list_physical_devices(device_type=None)[-1][-1]))

Notebook run in 9.8 seconds on CPU
